**Разбивка на подзадачи**:
- подготовка данных
- методика расчета средних охватов по станции (15мин, день, неделя, месяц)
- методика расчета суммарного рейтинга GRP
- методика расчета Reach 1+ для однодневной рекламной кампании (РК)  
- методика Reach 1+ для расчета РК по нескольким дням внутри одной недели (пересечение однодневных охватов)
- методика расчета частотного охвата

In [1]:
import pandas as pd
import math
import numpy as np
import matplotlib
import seaborn as sns
import scipy as sc
import functions
import warnings
warnings.filterwarnings ( action='ignore')

# 1. Подготовка данных

Исходные данные представлены в виде 2 таблиц: в одной закодированы дневники слушания по каждому дню недели в разрезе 15 минуток, во второй закодированы соц-дем профиль, в том числе ответы на вопросы о частоте слушания той или иной радиостанции.  
Исследование проводилось в г. Москва в 1 кв 2021г методом телефонного опроса Day-After-Recall, и включает результаты опроса 15 138 респондентов.   
Для расчетов будем использовать не полные данные, а выборку по одной станции - Авторадио (код 101)    
Также таблица с соц-демом содержит 500 с лишним колонок, поэтому для расчетов возьмем только блок с ответами на вопросы о частоте слушания Авторадио.

In [3]:
# Дневники слушания (сведенные в одну таблицу)
diary = pd.read_csv ('./src/data/diary.csv', sep=',')
diary

,Member_nr,day,Chl_id,weights,1,2,3,4,5,6,...,87,88,89,90,91,92,93,94,95,96
0,1000000005,5,101,1.0603,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,1000000018,1,101,0.8696,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,1000000048,7,101,1.5465,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,1000000049,1,101,1.1060,0.0,0.0,0.0,0.0,1.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,1000000082,6,101,0.9800,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1167,1000015018,4,101,0.7661,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1168,1000015029,1,101,0.9223,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1169,1000015101,6,101,2.1077,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1170,1000015118,6,101,0.7470,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


Описание колонок:  
**Member_nr** - уникальный ID респондента  
**day** - закодированный день недели слушания  
**Chl_id** - код радиостанции   
**weights** - веса респондентов (в методиках часто используется другое обозначение *projection factor*), это количество людей в генеральной совокупности соответствующее одной весовой единице в опросе. Проще говоря, это поправочный коэффициент, который позволяет выровнять распределение основных соц-дем характеристик (пол, возраст, размер семьи) между выборкой и ген. совокупностью.
**1...96** - номера слотов (15 минутных интервалов в сутках), в которых указан рейтинг слушания:  
- ноль, если респондент не слушал ни одной радиостанции в данном 15-мин интервале
- единица деленная на количество станций, которые слушал респондент в данном 15-мин интервале, в противном случае    

In [4]:
# Выборка из соц-дем таблицы
rt = pd.read_csv ('./src/data/data.csv', sep=';')
rt

,ID,w,day_list,week_list,month_list
0,1000000001,0.6037,0,0,0
1,1000000002,1.1176,0,0,0
2,1000000003,0.8871,0,0,0
3,1000000004,1.0779,0,0,0
4,1000000005,1.0603,1,7,1
...,...,...,...,...,...
15133,1000015134,1.7235,1,7,1
15134,1000015135,0.8765,0,6,1
15135,1000015136,0.8866,0,0,0
15136,1000015137,0.8601,0,0,1


Описание колонок:  
**ID** - уникальный ID респондента  
**w** - веса респондентов  
**day_list** - индикатор суточного слушания "Авторадио"   
**week_list** - индикатор недельного слушания "Авторадио" (количество дней недели)   
**month_list** - индикатор месячного слушания "Авторадио" 

В качестве бенчмарка используем 3 медиаплана, рассчитанные в SuperNova MediaScope по следующим вводным:    
**База данных:** Radio Index - Moscow. January - March 2021  
**Размер генеральной совокупности (тыс.):** 10 973,3  
**Размер целевой группы (тыс.):** 10 973,3  
**Выборка:** 15 138    
**Целевые медиа:** Авторадио  
**МП1** - 2 выхода в час, 4 выхода в день, с 10 до 11, с 12 до 13, с 14 до 15, с 18 до 19 часов, период: понедельник (одна неделя), итого - 8 выходов  
**МП2** - 2 выхода в час, 4 выхода в день, с 10 до 11, с 12 до 13, с 14 до 15, с 18 до 19 часов, период: понедельник-пятница  (одна неделя), итого - 40 выходов  
**МП3** - 4 выхода в час, 3 выхода в день, с 15 до 18 часов, период: понедельник-воскресенье  (четыре недели), итого - 336 выходов  

In [5]:
mp1 = pd.read_csv ('./src/data/mp1.csv', sep=';').dropna()
mp2 = pd.read_csv ('./src/data/mp2.csv', sep=';').dropna()
mp3 = pd.read_csv ('./src/data/mp3.csv', sep=';').dropna()

# посмотрим на mp1
mp1

,Показатель,Авторадио
0,GI,555.88
1,Frequency,1.82
2,Index T/U Reach,100.00
3,GRP,5.07
4,TRP,5.07
5,RP Index,100.00
6,Spots,8.00
7,Reach 1+,306.13
8,Reach 2+,120.83
9,Reach 3+,58.34


Здесь много показателей, но базовые это **Reach** и **GRP/TRP**, остальные являются производными от них в той или иной степени

На последнем этапе подготовки зафиксируем переменные среды, так как вызовы функций настроены на входные данные, которые должны содержать разные станции и разное расписание выходов, а мы работаем с зафиксированными вводными, чтобы каждый раз не передавать эти данные, запишем их в константы

In [6]:
CH = 101 # код Авторадио
MP1_D = [1] # номер дня недели выхода ролика
MP2_D = [1,2,3,4,5]
MP3_D = [1,2,3,4,5,6,7]
MP1_SCH = [10,12,14,18] # часы выходов
MP2_SCH = MP1_SCH
MP3_SCH = [15,16,17]
MP1_N = 2 # количество выходов в час
MP2_N = MP1_N
MP3_N = 4

# 2.  Подсчет аудиторий станции (15 мин, день, неделя, месяц)

Расчет рейтинга 15 минутки производится по формуле:
$\sum_{i = 1}^{n}\dfrac{w_i*t_i}{Day\_Aud}$, где 
- $n$ - количество респондентов, имеющих записи в дневнике слушания в выбранный день
- $w_i$ веса всех респондентов, присутствующих в дневнике слушания в выбранный день
- $t_i$ рейтинг слушания выбранной радиостанции в интервале 15 минут (значение от нуля до единцы, в зависимости от количества слушаемых станций) 
- $Day\_Aud$ сумма весов респондентов, опрошенных в выбранный день  
Последнее значение в расчетах можно принять за константу равную $\dfrac{15\ 138}{7}$, так как волна опроса соответствующим образом была равномерно распределена по дням недели

In [7]:
# Итоговая функция для подсчета рейтинга 15-минутки
# считаем на примере: понедельник, Авторадио, интервал: 14:00 - 14:15 
# нумерация 15-минуток в дневниках слушания начинается с 5:00 утра, так что времени 14:00 соответствует 37 слот
round (functions.reach_slot(MP1_D[0], CH ,'37')*100, 2)

0.91

Теперь подсчитаем показатель **AQH** (средний рейтинг 15-минутки) для Авторадио, по методике для этого надо рассчитать рейтинги всех 15 минуток по всем 7 дням недели и усреднить их.  
При этом в отчете MediaScope за 1 кв 2021г, значения AQH даны для диапазона 06:00-24:00, поэтому нам также нужны значения только для этого интервала времени 

In [8]:
# AQH Авторадио
res=[]
for day in range (1,8):
    for slot in range (5, 77):
        res.append (functions.reach_slot ( day, CH, str(slot)))
print ('AQH Авторадио -', round (np.array(res).mean() ,3) *100) 

AQH Авторадио - 0.5


Сравниваем с бенчмарком - все совпало
<img src='./src/images/AQH.png'>

Теперь перейдем к расчету средних рейтингов: **Reach Daily**, **Reach Weekly**, **Reach Monthly**  
Они определяются очень просто - как доля положительно ответивших на соответствующий вопрос по отношению к общему количеству опрошенных, с учетом весовых коэффициентов.
Получим соотвествующие значения для Авторадио и сравним их с бенчмарком (см. скрин выше)

In [ ]:
print ('Средний дневной охват "Авторадио", Daily Reach-', round(np.dot(rt['day_list'],rt['w'])/15138 *100, 2 )) 
print ('Средний недельный охват "Авторадио", Weekly Reach -', round(np.dot(np.where (rt['week_list']==0,0,1),rt['w'])/15138*100,1))
print ('Средний месячный охват "Авторадио", Monthly Reach -', round(np.dot(rt['month_list'],rt['w'])/15138 *100, 2 )) 

Расчеты верны, можем смело опираться на эти формулы в дальнейшем

In [9]:
# 3. Расчет GRP

Переходим к расчетам медиапоказателей РК, начнем с самого простого: подсчету **GRP/TRP**.  
Итоговая формула для расчета GRP выглядит следующим образом:  
$GRP = \sum_{d}\sum_{i = 1}^{k} Reach\_slot_i*\dfrac{n_i}{4}$, где
- $d$ - дни выходов ролика в течении РК
- $k$ - количество слотов в которых есть выходы ролика
- $Reach\_slot$ - рейтинг 15-минутки
- $n$ - количество выходов ролика в час  
По сути это просто сумма рейтингов 15-минуток, с поправкой на количество выходов в час. При медиапланировании РК на радио минимальным периодом планирования является час и ролики равномерно распределяются по сетке выходов, соответственно, если количество выходов в час равно 1, то рассматривая эту ситуацию с точки зрения теории вероятности, мы понимаем что, с вероятностью 0,25 выход ролика попадет в любой из 4 слотов, итоговую вероятность по каждому респонденту надо оценивать как 
сумму рейтингов слушания 15 минуток с коэффициентом 0,25. Если будет 2 выхода в час, то 1 ролик выйдет в первые полчаса (первые 2 слота часа), а второй ролик во вторые полчаса (3,4 слот часа). Соответственно, первый ролик с вероятностью 0,5 попадет в 1 слот, либо с вероятностью 0,5 во второй. Аналогично со вторым роликом и вероятностями попасть в 3 и 4 слоты. В итоге получается формула для расчета поправочного коэффициента $\dfrac{n}{4}$  

*Примечание*: поскольку мы изначально зафиксировали в качестве целевой аудитории всех респондентов, в нашем случае **GRP=TRP**.  На практике чаще всего работают с целевой аудиторией и необходимо также рассчитывать и **TRP**. Формула расчета **TRP** аналогична формуле расчета **GRP**, но рейтинги 15-минуток считаются только по целевой аудитории (то есть в числителе суммируются только веса респондентов из целевой аудитории, а в знаменателе в сумме задействованы только веса респондентов, принадлежащих целевой аудитории).  

Посчитаем GRP по формуле и сравним с данными из медиапланов

In [10]:
print ("Расчетное значение GRP: ", round (functions.grp (CH, MP1_D, MP1_SCH, MP1_N) ,2), ' | Значение МП 1: ' , mp1.to_dict (orient='records')[3])
print ("Расчетное значение GRP: ", round (functions.grp (CH, MP2_D, MP2_SCH, MP2_N) ,2), '| Значение МП 2: ' , mp2.to_dict (orient='records')[3])
print ("Расчетное значение GRP: ", round (functions.grp (CH, MP3_D, MP3_SCH, MP3_N) , 2), '| Значение МП 3: ' , mp3.to_dict (orient='records')[3] )

Расчетное значение GRP:  5.07  | Значение МП 1:  {'Показатель': 'GRP', 'Авторадио': 5.07}
Расчетное значение GRP:  25.95 | Значение МП 2:  {'Показатель': 'GRP', 'Авторадио': 25.95}
Расчетное значение GRP:  52.86 | Значение МП 3:  {'Показатель ': 'GRP', 'Авторадио': '211.44'}


Данные совпали, кроме последнего медиаплана, так как последний МП рассчитан на 4 недели, а **GRP** считается только в рамках одной недели. Для получения корректной цифры надо отдельно высчитать **GRP** по каждой неделе размещения и затем просуммировать их. Так как в нашем примере показатели РК не меняются от недели к неделе, то **GRP** по каждой неделе размещения идентичны и мы можем просто умножить **GRP** первой недели на 4.

In [11]:
print ("Расчетное значение GRP: ", round (functions.grp (CH, MP3_D, MP3_SCH, MP3_N)*4 ,2) , '| Значение МП 3: ' , mp3.to_dict (orient='records')[3] )

Расчетное значение GRP:  211.45 | Значение МП 3:  {'Показатель ': 'GRP', 'Авторадио': '211.44'}


Теперь все значения совпадают с бенчмарком

# 4. Расчет Reach 1+ 

Теперь переходим к расчету охватов РК на частоте 1+  
Для начала рассмотрим расчет охвата в какой-либо один конкретный день размещения.
В существующих методиках охват рассматривается с точки зрения теории вероятностей, то есть для каждого респондента рейтинг слушания рассматривается как определенная вероятность услышать выход ролика в конкретную 15-минутку, это можно иллюстрировать формулой вида:  
$p_{ik} = r_{ik}*\dfrac{n}{4}$ ,  
- $p_{ik}$ - вероятность для респондента $i$ услышать ролик хотя бы раз в $k$ 15-минутку  
- где $r_{ik}$ - рейтинг слушания респондента $i$ выбранной радиостанции в $k$ 15-минутку, 
- $n$ - количество выходов ролика в час  

Далее все эти вероятности по респонденту $i$ суммируются по формуле:  
$E_i = \sum_{s}\sum_{k = 1}^{m} p_{ik}$, 
- $m$ это количество 15-минуток размещения в течение дня,
- $s$ - станция/станции где размещается ролик в выбранный день

В итоге получается суммарная вероятность услышать ролик хотя бы 1 раз в день, если значение получилось больше 1, то мы принимаем $E_i=1$, затем по каждому респонденту мы считаем сумму произведений весов на показатель $E_i$ и делим на сумму весов респондентов, опрошенных в выбранный день:  
$Reach_{day}\space 1+= \sum_{i = 1}^{n}\dfrac{w_i*E_i}{Day\_Aud}$

*Примечание*: вообще, с точки зрения теории вероятностей, надо учитывать совместную вероятность наступления события, и корректнее расчет производить по формуле единица минус вероятность события "респондент ни разу не услышал ролик" $p_0 = 1-((1-p_{ik_1})*(1-p_{ik_2})...(1-p_{ik_m}))$, но на практике количество множителей в последней формуле достаточно большое, для того, чтобы итоговое произведение было близко к нулю, я предполагаю, что именно поэтому в методиках используется простое суммирование вероятностей для упрощения расчетов

Проверим корректность формулы на примере первого медиаплана

In [12]:
print ("Расчетное значение Reach 1+: ", round (functions.reach_day (CH, MP1_D[0], MP1_SCH, MP1_N)*100, 3),
       '| Значение из МП 1: ' , mp1.to_dict (orient='records')[12] )

Расчетное значение Reach 1+:  2.785 | Значение из МП 1:  {'Показатель': 'Reach%  1+', 'Авторадио': 2.79}


Формула работает.  
Теперь более сложная задача - посчитать охват на частоте 1+ для нескольких дней.  
По науке мы должны сложить охваты по всем дням и вычесть всевозможные пересечения аудиторий. Но у нас нет данных по пересечениям аудиторий, так как в каждый конкретный день недели опрашиваются отдельные группы респондентов и записи в дневнике слушания ограничены только одним днем слушания. Поэтому нужно использовать формулу вероятности совместных событий, которая была приведена чуть выше. У нас есть данные по недельному охвату радиостанции, используя его в качестве базы для расчетов получаем следующую формулу:  
$RCR\ =WR*(1-(1-\dfrac{Reach_{d1}}{WR})\ast(1-\dfrac{Reach_{d2}}{WR})\ast(1-\dfrac{Reach_{d3}}{WR}).....\ast(1-\dfrac{Reach_{d7}}{WR}))$, где  
- $RCR$ - охват РК за несколько дней    
- $Reach_{dn}$ - охват РК в конкретный день $n$ (*Reach 1 +*)      
- $WR$ - недельный охват станции (*Weekly Reach*)

Попробуем посчитать охват для второго медиаплана, в котором период проведения РК: понедельник - пятница

In [13]:
# получаем недельный охват
WR = np.dot(np.where (rt['week_list']==0,0,1),rt['w'])/15138 
res=[]
#вычисляем RCR
for day in MP2_D:
    res.append(1-functions.reach_day (CH, day, MP2_SCH, MP2_N)/WR) 
RCR = (1-np.array(res).prod())*WR*100
print ('Расчетное значение Reach 1+: ' ,  round (RCR, 2),
       '| Значение из МП 2: ' , mp2.to_dict (orient='records')[12] 
      ) 

Расчетное значение Reach 1+:  12.15 | Значение из МП 2:  {'Показатель': 'Reach%  1+', 'Авторадио': 12.27}


Расчетный охват меньше ожидаемого (хоть и в пределах 5% интервала допустимого отклонения). Значит применяется какой-то повышающий коэффициент, для понимания нужно рассмотреть extreme case: что, если по каждому дню неделю РК набирает максимально возможное количество выходов во всех слотах на протяжении всего дня? По логике это будет означать, что мы охватим всех возможных слушателей этой радиостанции за неделю и охват РК будет равен недельному охвату станции.  Однако, если мы рассчитаем на таких вводных медиаплан по нашей формуле, мы увидим, что расчетные цифры будут меньше

In [14]:
# определяем максимальные значения РК: все дни недели, все 15-минутки дня, по 1 выходу каждые 5 минут (технический максимум станций)
mp_full_day = list (np.arange(1,8))
mp_full_sch = list (np.arange (0,24))
WR = np.dot(np.where (rt['week_list']==0,0,1),rt['w'])/15138 
res=[]
#вычисляем RCR_max 
for day in mp_full_day:
    res.append(1-functions.reach_day (CH, day, mp_full_sch, 12)/WR) 
RCR_max = (1-np.array(res).prod())*WR*100
print ('Расчетное значение RCR_max' , round (RCR_max,2), ', бенчмарк Weekly Reach', round (WR*100, 2))

Расчетное значение RCR_max 24.15 , бенчмарк Weekly Reach 25.21


Для корректировки значения и нужен повышающий коэффициент, самое простое рассчитать поправочный коэффициент как $\dfrac{WR}{RCR_{max}}$ (в нашем случае это **1.0428**) и домножать на него полученные расчетные значения RCR, но тогда мы столкнемся с другими трудностями: на малых значениях формула будет отрабатывать некорректно. Предположим, что мы в первый день охватили 1% от недельной аудитории и во второй день охватили 1% аудитории, по формуле $1-(1-0.01)*(1-0.01)$ получаем RCR равный **1.99%**, однако домножив его на **1.0428** мы получим охват больше, чем **2%**, то есть больше максимально возможного значения.  
Соответственно, повышающий коэффициент должен варьироваться в зависимости от того насколько значение RCR близко к максимуму.
Для этого вводится еще один показатель, отражающий насколько сильно охват РК по дням соотносится с дневными максимумами:  
$Pow = \dfrac{\sum_{n = 1}^{N}Reach_{d\_n}}{\sum_{d = 1}^{7}Reach_{d\_max}}$, где 
- $Reach_{d\_n}$   - охват РК в день $n$ 
- $Reach_{d\_max}$ - максимально возможный охват в день недели $d$ 

Значение в знаменателе является константой для каждой станции, поскольку зависит только от станции, а не от параметров РК  

Финальная формула для расчета повышающего коэффициента имеет вид:  
$RCF\ =\ (\dfrac{WR}{RCR_{max}} -1)*Pow+1$  

А итоговая формула для расчета охвата на частоте 1+ по нескольким дням:  
$MixCR\ =\ WR\ast RCR\ast RCF$

Снова проверим на втором медиаплане

In [15]:
print ('Расчетное значение Reach 1+: ' ,  round (functions.mix_reach (CH, MP2_D, MP2_SCH, MP2_N)*100, 3),
       '| Значение из МП 2: ' , mp2.to_dict (orient='records')[12] 
      ) 

Расчетное значение Reach 1+:  12.275 | Значение из МП 2:  {'Показатель': 'Reach%  1+', 'Авторадио': 12.27}


Удалось значительно сократить отклонение от бенчмарка.
Попробуем посчитать охват для нескольких недель, используя похожую логику, в качестве предельного охвата теперь будет выступать **Monthly Reach**, в качестве дневных охватов РК - недельные охваты РК 

In [16]:
# получаем Monthly Reach
MR = np.dot(rt['month_list'],rt['w'])/15138
res=[]
#вычисляем MixCR для каждой недели медиаплана 3 (они одинаковы, поэтому просто 4 раза складываем MixCR)
for week in [1,2,3,4]:
    res.append(1-functions.mix_reach (CH, MP3_D, MP3_SCH, MP3_N)/MR) 
RCR = (1-np.array(res).prod())*MR*100
print ('Расчетное значение Reach 1+: ' ,  round (RCR, 2),
       '| Значение из МП 3: ' , mp3.to_dict (orient='records')[12] 
      ) 

Расчетное значение Reach 1+:  30.07 | Значение из МП 3:  {'Показатель ': 'Reach%  1+', 'Авторадио': '22.28'}


In [17]:
#максимальный охват на протяжении 4 недель
res=[]
for week in [1,2,3,4]:
    res.append(1-functions.mix_reach (CH, mp_full_day, mp_full_sch, 12)/MR) 
RCR_max = (1-np.array(res).prod())*MR*100
print ('Расчетное значение RCR_max' , round (RCR_max,2), ', бенчмарк Monthly Reach', round (MR*100, 2))

Расчетное значение RCR_max 34.37 , бенчмарк Monthly Reach 34.55


Расчетный показатель значительно выше, чем в бенчмарке, при том, что при рассмотрении extreme case формула вполне корректно работает, не требуя никаких значительных поправочных коэффициентов (отклонение меньше 1%):   

Такая разница в случае медиаплана номер 3 может объясняться тем, что у MediaScope есть данные исследований о размерах недельных пересечений аудиторий внутри месяца, либо тем, что для расчета пересечения аудиторий внутри месяца используется более сложная и многоступенчатая методика, чем использованная для расчета пересечений внутри недели. В любом случае подбор этой методики выглядит труднорешаемой задачей, особенно в случае использования данных из каких-то сторонних исследованиях

# 5. Частотное распределение охвата

Последний основной медиапоказатель - распределение охвата 1+ по частотам. Для расчета охвата на конкретной частоте 1,2,3 и т.д. используется либо отрицательное биномиальное распределение (**NBD**) c вариантом параметризации через *a* и *k*, либо распределение Пуассона.  
Критерий выбора: если значение $C<-1$, то используется NBD, иначе распределение Пуассона. $C$ определяется по формуле:
$C\ =\ \ \dfrac{TRP}{100\ast ln(1-Reach\ 1+)}$

**Чуть подробнее про NBD**: вероятность охвата на конкретной частоте моделируется как вероятность получить определенное количество "успехов" в серии испытаний Бернулли до заданного количества "неудач". В качестве "успеха" выступает событие, что ролик услышали. Обозначая вероятность "успеха" через $\dfrac{1}{(1+a)}$, а обратную вероятность через $\dfrac{a}{(1+a)}$ получаем формулу для расчета $Reach$ на частоте $i$:  
$Reach_i\ =\dfrac{Г(k+i)}{Г(k)*Г (i+1)}*(\dfrac{1}{1+a})^k*(\dfrac{a}{1+a})^i$, где
- $Г$ - значение гамма-функции (в данном случае выступает обобщением факториала для непрерывных значений) 
- $a$ и $k$ параметры распределения, которые рассчитываются на основе TRP и Reach 1+

Для получения значений $a$ и $k$ используются следующая система уравнений:   
a) $1-Reach1+ = Reach_0 = (\dfrac{1}{1+a})^k$  
б) $E(X) = \dfrac{TRP}{100} = a*k$  
Первое уравнение - это частный случай формулы при нулевом количестве "успехов", то есть охват на частоте 0, второе уравнение это формула мат.ожидания NBD, которое приравнивается к TRP  

In [18]:
#вычисляем reach и grp по МП1 и МП2
reach_mp_1, grp_mp_1 = functions.mix_reach (CH, MP1_D, MP1_SCH, MP1_N), functions.grp (CH, MP1_D, MP1_SCH, MP1_N)/100
reach_mp_2, grp_mp_2 = functions.mix_reach (CH, MP2_D, MP2_SCH, MP2_N), functions.grp (CH, MP2_D, MP2_SCH, MP2_N)/100
print ('МП1, use NBD - ' , functions.crit (reach_mp_1, grp_mp_1 )<-1)
print ('МП2, use NBD - ' , functions.crit (reach_mp_2, grp_mp_2 )<-1)

МП1, use NBD -  True
МП2, use NBD -  True


В обоих случаях применяем NBD

In [19]:
functions.reach_NBD (reach_mp_1, grp_mp_1) 

{'Reach 1+ ': 306.15,
 'Reach 2+ ': 120.83,
 'Reach 3+ ': 58.34,
 'Reach 4+ ': 30.6,
 'Reach 5+ ': 16.82,
 'Reach% 1+ ': 2.79,
 'Reach% 2+ ': 1.1,
 'Reach% 3+ ': 0.53,
 'Reach% 4+ ': 0.28,
 'Reach% 5+ ': 0.15}

In [20]:
#сверяем с МП1
mp1.iloc[7:,:]

,Показатель,Авторадио
7,Reach 1+,306.13
8,Reach 2+,120.83
9,Reach 3+,58.34
10,Reach 4+,30.61
11,Reach 5+,16.82
12,Reach% 1+,2.79
13,Reach% 2+,1.10
14,Reach% 3+,0.53
15,Reach% 4+,0.28
16,Reach% 5+,0.15


In [21]:
functions.reach_NBD (reach_mp_2, grp_mp_2) 

{'Reach 1+ ': 1347.01,
 'Reach 2+ ': 624.74,
 'Reach 3+ ': 340.93,
 'Reach 4+ ': 199.34,
 'Reach 5+ ': 121.2,
 'Reach% 1+ ': 12.28,
 'Reach% 2+ ': 5.69,
 'Reach% 3+ ': 3.11,
 'Reach% 4+ ': 1.82,
 'Reach% 5+ ': 1.1}

In [22]:
#сверяем с МП2
mp2.iloc[7:,:]

,Показатель,Авторадио
7,Reach 1+,1346.52
8,Reach 2+,624.68
9,Reach 3+,340.99
10,Reach 4+,199.43
11,Reach 5+,121.28
12,Reach% 1+,12.27
13,Reach% 2+,5.69
14,Reach% 3+,3.11
15,Reach% 4+,1.82
16,Reach% 5+,1.11


Расчетные значения с незначительным отклонением от бенчмарка. Для третьего МП нет расчетного значения Reach 1+, но для проверки корректности формулы NBD можно воспользоваться значением из бенчмарка 

In [23]:
reach_mp_3, grp_mp_3 = float(mp3.to_dict (orient='records')[12].get('Авторадио'))/100, functions.grp (CH, MP3_D, MP3_SCH, MP3_N)*4/100
functions.reach_NBD (reach_mp_3, grp_mp_3) 

{'Reach 1+ ': 2444.85,
 'Reach 2+ ': 1830.39,
 'Reach 3+ ': 1511.49,
 'Reach 4+ ': 1298.48,
 'Reach 5+ ': 1140.33,
 'Reach% 1+ ': 22.28,
 'Reach% 2+ ': 16.68,
 'Reach% 3+ ': 13.77,
 'Reach% 4+ ': 11.83,
 'Reach% 5+ ': 10.39}

In [24]:
#сверяем с МП3
mp3.iloc[7:,:]

,Показатель,Авторадио
7,Reach 1+,2 444.47
8,Reach 2+,1 830.13
9,Reach 3+,1 511.29
10,Reach 4+,1 298.32
11,Reach 5+,1 140.20
12,Reach% 1+,22.28
13,Reach% 2+,16.68
14,Reach% 3+,13.77
15,Reach% 4+,11.83
16,Reach% 5+,10.39


С частотным распределением все ок

Последнее, что стоит упомянуть, это как считать частотное распределение Пуассона, для этого используется формула:
$Reach\ i\ =\ \dfrac{\lambda^i\ast e^{-\lambda}}{i!}$, где $\lambda=\dfrac{TRP}{100}$  
На практике эта необходимость возникает крайне редко, так как крайне редко встречаются РК одновременно и короткие по времени и с малым охватом

# Заключение

В финальной части соберем все, что мы умеем корректно считать в одну универсальную функцию, которая будет рассчитывать медиапоказатели по заданным параметрам РК и выводить их в датафрейм

In [25]:
functions.mp (CH, MP1_D, MP1_SCH, MP1_N)

,Расчет
GI,555.88
Frequency,1.82
Index T/U Reach,100.00
GRP,5.07
TRP,5.07
RP Index,100.00
Spots,8.00
Reach 1+,306.15
Reach 2+,120.83
Reach 3+,58.34


При объединении датафреймов можно рассчитать точные отклонения от бенчмарка

In [ ]:
df1 = mp1.join (functions.mp (CH, MP1_D, MP1_SCH, MP1_N).reset_index(drop=True))
df1['Diff%'] =  round ( (df1['Авторадио'] - df1['Расчет']) / df1['Авторадио']*100 ,2)
df1

In [ ]:
df2 = mp2.join (functions.mp (CH, MP2_D, MP2_SCH, MP2_N).reset_index(drop=True))
df2['Diff%'] =  round ((df2['Авторадио'] - df2['Расчет']) / df2['Авторадио']*100 , 2)
df2

*Примечание*: все расчеты сделаны на примере одной станции, но алгоритм легко масштабируется для случая РК по нескольким станциям. Для этого просто нужно агрегировать аудиторию нескольких станций в аудиторию одной большой метастанции и дальнейшие расчеты вести с аудиторией этой одной большой станции. К примеру, нам нужны сводные медиапоказатели для РК на "Авторадио" (**101** в дневниках слушания) и "Европа +" (**105** в дневниках слушания), тогда мы просто в дневниках слушания заменяем значения **101** и **105** на, например, **201** и дальше делаем все расчеты для станции с кодом **201**